In [8]:
##授權允許Colab存取個人雲端資料夾
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [9]:
#flask api
from flask import Flask, request
import base64
import io

!pip install yolov5
import torch
from PIL import Image
from yolov5 import detect

app = Flask(__name__)

# 存取控制清單
control_list = {"car": 0, "person": 1, "gun": 2, "knife": 2, "blood": 3}

# 使用者等級，默認為0
user_level = 0

# 使用者ID，默認為user
user_id = "user"

@app.route('/api/objects', methods=['POST'])

#使用Yolov5初始模型完成預測
def detect(image):
    # 調用函數獲取預測结果
    model = torch.hub.load('ultralytics/yolov5', 'yolov5s')
    results = model(image)

    # 提取物件名稱和座標
    pred_classes = results.pandas().xyxy[0]['name']
    pred_boxes = results.pandas().xyxy[0][['xmin', 'ymin', 'xmax', 'ymax']]

    # 建構物件清單
    object_list = []
    for i in range(len(pred_classes)):
        obj = {
          'name': pred_classes[i],
          'xmin': pred_boxes.iloc[i]['xmin'],
          'ymin': pred_boxes.iloc[i]['ymin'],
          'xmax': pred_boxes.iloc[i]['xmax'],
          'ymax': pred_boxes.iloc[i]['ymax']
      }
    return object_list.append(obj)

# 篩選出使用者禁止觀看的物件清單，並輸出物件序數
def filter_forbidden_objects(user_level, object_list, control_list):
    forbidden_object_indices = []

    for i, obj in enumerate(object_list):
        object_name = obj['name']
        object_level = control_list.get(object_name, 0)

        if user_level < object_level:
            forbidden_object_indices.append(i)

    return forbidden_object_indices

#取得禁止物件清單，並通過序數取得物件座標
def get_forbidden_object_coordinates(forbidden_object_indices, object_list):
    forbidden_object_coordinates = []

    for index in forbidden_object_indices:
        obj = object_list[index]
        coordinates = {
            'xmin': obj['xmin'],
            'ymin': obj['ymin'],
            'xmax': obj['xmax'],
            'ymax': obj['ymax']
        }
        forbidden_object_coordinates.append(coordinates)

    return forbidden_object_coordinates

def detect_objects_api():
    # 獲取請求中的參數
    user_id = request.form['user_id']
    user_level = int(request.form['user_level'])
    image_base64 = request.form['image']

    # 將base64格式的截圖轉換為圖像
    image_bytes = base64.b64decode(image_base64)
    image = Image.open(io.BytesIO(image_bytes))

    #調用物件辨識方法
    object_list = detect(image)

    # 調用過濾方法
    forbidden_object_indices = filter_forbidden_objects(user_level, object_list, control_list)

    # 獲取禁止物件座標清單，將過濾過的物件座標作為HTTP回應回傳
    forbidden_object_coordinates = get_forbidden_object_coordinates(forbidden_object_indices, object_list)
    return {'forbidden_objects': forbidden_object_coordinates}

if __name__ == '__main__':
    app.run()

 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
